In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_7071/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [9]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2_sup")
        # self.discard_below_average(sort_by="mse_sup")
        # self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results_06.xlsx", index=True)
        display(self.results_df)


In [10]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_5_8_24,0.989911,0.854977,0.784679,0.985816,0.956103,0.014320,0.205839,0.033696,0.004178,0.018937,0.196784,0.119668,1.000140,0.124762,3526.492146,5670.494722,"Hidden Size=[40, 39], regularizer=0.02, learni..."
1,model_5_8_23,0.989831,0.854903,0.786183,0.986183,0.956501,0.014433,0.205944,0.033461,0.004070,0.018766,0.199129,0.120138,1.000141,0.125253,3526.476445,5670.479021,"Hidden Size=[40, 39], regularizer=0.02, learni..."
2,model_5_9_0,0.989993,0.854888,0.935895,0.953563,0.968507,0.014204,0.205965,0.011160,0.005456,0.008308,0.198233,0.119181,1.000138,0.124254,3526.508458,5670.511034,"Hidden Size=[40, 39], regularizer=0.02, learni..."
3,model_5_8_22,0.989748,0.854825,0.787650,0.986542,0.956890,0.014551,0.206054,0.033231,0.003964,0.018598,0.201515,0.120627,1.000142,0.125762,3526.460225,5670.462801,"Hidden Size=[40, 39], regularizer=0.02, learni..."
4,model_5_9_1,0.990068,0.854795,0.932283,0.952397,0.967055,0.014097,0.206098,0.011789,0.005593,0.008691,0.199671,0.118730,1.000137,0.123785,3526.523613,5670.526189,"Hidden Size=[40, 39], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,model_27_8_1,0.978298,0.823946,0.935362,0.686811,0.941283,0.030803,0.249883,0.102790,0.015487,0.059139,0.452672,0.175509,1.000932,0.182981,1172.960258,1883.564864,"Hidden Size=[26, 18], regularizer=0.0, learnin..."
1477,model_4_3_0,0.989602,0.823941,0.993036,0.956623,0.988318,0.014759,0.249890,0.021703,0.020452,0.021078,0.682053,0.121485,1.000144,0.126657,3526.431856,5670.434432,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
1478,model_32_7_14,0.993610,0.823935,0.985848,0.951749,0.976320,0.009070,0.249898,0.019584,0.033397,0.026491,0.423649,0.095236,1.000274,0.099290,1175.405599,1886.010205,"Hidden Size=[26, 18], regularizer=0.5, learnin..."
1479,model_27_8_0,0.978292,0.823906,0.936097,0.692631,0.942006,0.030812,0.249939,0.101621,0.015200,0.058410,0.452153,0.175533,1.000932,0.183006,1172.959715,1883.564321,"Hidden Size=[26, 18], regularizer=0.0, learnin..."
